# DATA WRANGLING PROJECT 
---------------------------------------
## WeRateDogs Twitter Data

#### 1. Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import tweepy